<a href="https://colab.research.google.com/github/friedelj/ML540/blob/main/JFriedel_Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#J Friedel  Assignment 2.1                      USD 540 Machine Learning Operations                                 26 May 2025

# 1. Ingest homework data into data lake.

# Setup All Workshop Dependencies FROM 01_setup_env_01_Setup_Dependencies

In [ ]:
!python --version

In [ ]:
!pip list

In [ ]:
#Pip
!pip install --disable-pip-version-check -q pip --upgrade > /dev/null
!pip install --disable-pip-version-check -q wrapt --upgrade > /dev/null

In [ ]:
# AWS CLI and AWS Python SDK (boto3)
!pip install --disable-pip-version-check -q awscli boto3

In [ ]:
# SageMaker
!pip install --disable-pip-version-check -q sagemaker
!pip install --disable-pip-version-check -q smdebug
!pip install --disable-pip-version-check -q sagemaker-experiments

In [ ]:
# PyAthena
!pip install --disable-pip-version-check -q PyAthena

In [ ]:
# Zip
!conda install -y zip

In [ ]:
# Matplotlib
!pip install --disable-pip-version-check -q matplotlib

In [ ]:
# Seaborn
!pip install --disable-pip-version-check -q seaborn

In [ ]:
# Summarize
!python --version

In [ ]:
!pip list

In [ ]:
setup_dependencies_passed = True
%store setup_dependencies_passed

In [ ]:
%store

# END of 01_setup_env_01_Setup_Dependencies

# START 01_setup_env_02_Create_S3_Bucket

In [ ]:
# Create S3 Bucket
import boto3
import sagemaker

session = boto3.session.Session()
region = session.region_name
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

s3 = boto3.Session().client(service_name="s3", region_name=region)

In [ ]:
setup_s3_bucket_passed = False
print("Default bucket: {}".format(bucket))

In [ ]:
# Verify S3_BUCKET Bucket Creation
from botocore.client import ClientError

response = None

try:
    response = s3.head_bucket(Bucket=bucket)
    print(response)
    setup_s3_bucket_passed = True
except ClientError as e:
    print("[ERROR] Cannot find bucket {} in {} due to {}.".format(bucket, response, e))

In [ ]:
%store setup_s3_bucket_passed

In [ ]:
%store

# END  01_setup_env_02_Create_S3_Bucket

# BEGIN 02_setup_datalake

In [ ]:
# Check Pre-Requisites from an earlier notebook
%store -r setup_dependencies_passed
try:
    setup_dependencies_passed
except NameError:
    print("+++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL NOTEBOOKS IN THE SETUP FOLDER FIRST. You are missing Setup Dependencies.")
    print("+++++++++++++++++++++++++++++++")
print(setup_dependencies_passed)

In [ ]:
%store -r setup_s3_bucket_passed
try:
    setup_s3_bucket_passed
except NameError:
    print("+++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL NOTEBOOKS IN THE SETUP FOLDER FIRST. You are missing Setup S3 Bucket.")
    print("+++++++++++++++++++++++++++++++")
print(setup_s3_bucket_passed)

In [ ]:
if not setup_dependencies_passed:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL NOTEBOOKS IN THE SETUP FOLDER FIRST. You are missing Setup Dependencies.")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
if not setup_s3_bucket_passed:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL NOTEBOOKS IN THE SETUP FOLDER FIRST. You are missing Setup S3 Bucket.")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

import boto3
import sagemaker
import pandas as pd

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

In [ ]:
# Set S3 Destination Location (Our Private S3 Bucket)
s3_private_path_csv = "s3://{}/assmt2-ds/csv".format(bucket)
print(s3_private_path_csv)

In [ ]:
%store s3_private_path_csv

In [ ]:
# Copy Data From the Public S3 Bucket to Private S3 Bucket in this Account
!aws s3 cp "dataset.csv" $s3_private_path_csv/

In [ ]:
# List Files in our Private S3 Bucket in this Account
print(s3_private_path_csv)

In [ ]:
!aws s3 ls $s3_private_path_csv/

In [ ]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/sagemaker-{}-{}/assmt2-ds/?region={}&tab=overview">S3 Bucket</a></b>'.format(
            region, account_id, region
        )
    )
)

# Found in s3://sagemaker-us-east-1-244989531891/assmt2-ds/tsv/dataset.csv

In [ ]:
# Store Variables for the Next Notebooks
%store

# END 02_setup_datalake

# 2.Set up the Athena query engine

In [ ]:
import boto3
import sagemaker

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

ingest_create_athena_table_csv_passed = False

%store -r ingest_create_athena_db_passed

try:
    ingest_create_athena_db_passed
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL PREVIOUS NOTEBOOKS.  You did not create the Athena Database.")
    print("++++++++++++++++++++++++++++++++++++++++++++++")

In [ ]:
print(ingest_create_athena_db_passed)

In [ ]:
if not ingest_create_athena_db_passed:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL PREVIOUS NOTEBOOKS.  You did not create the Athena Database.")
    print("++++++++++++++++++++++++++++++++++++++++++++++")
else:
    print("[OK]")

In [ ]:
%store -r s3_private_path_csv

try:
    s3_private_path_csv
except NameError:
    print("*****************************************************************************")
    print("[ERROR] PLEASE RE-RUN THE PREVIOUS COPY TSV TO S3 NOTEBOOK ******************")
    print("[ERROR] THIS NOTEBOOK WILL NOT RUN PROPERLY. ********************************")
    print("*****************************************************************************")

print(s3_private_path_csv)

In [ ]:
# Create Athena Table from Local CSV Files
# Import PyAthena
from pyathena import connect

# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

# Set Athena parameters
database_name = "dsAssmt2"
table_name_csv = "assmt2_csv"

conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [ ]:
# Manually create the Athena database if it doesn't exist
create_db_statement = "CREATE DATABASE IF NOT EXISTS dsAssmt2"
cursor = conn.cursor()
cursor.execute(create_db_statement)
print("[OK] Created database dsAssmt2 (or already exists)")

In [ ]:
# SQL statement to execute
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         track_id string,
         artists string,
         album_name string,
         track_name string,
         popularity int,
         duration_ms string,
         explicit boolean,
         danceability decimal(4,3),
         energy decimal(5,4),
         key int,
         loudness decimal(5,3),
         mode int,
         speechiness decimal(5,4),
         acousticness decimal(5,4),
         instrumentalness float,
         liveness decimal(5,4),
         valence decimal(5,4),
         tempo decimal(6,3),
         time_signature int,
         track_genre string
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\\n' LOCATION '{}'
TBLPROPERTIES ('compressionType'='gzip', 'skip.header.line.count'='1')""".format(
    database_name, table_name_csv, s3_private_path_csv
)

print(statement)

In [ ]:
import pandas as pd

pd.read_sql(statement, conn)

In [ ]:
# Verify The Table Has Been Created Succesfully
statement = "SHOW TABLES in {}".format(database_name)

df_show = pd.read_sql(statement, conn)
df_show.head(5)

In [ ]:
# Sample query to preview data from the table
query = f"SELECT * FROM {database_name}.{table_name_csv} LIMIT 10"
df_preview = pd.read_sql(query, conn)
df_preview

In [ ]:
# SQL statement to execute
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         item_nunber int,
         track_id string,
         artists string,
         album_name string,
         track_name string,
         popularity int,
         duration_ms string,
         explicit int,
         danceability decimal(4,3),
         energy decimal(5,4),
         key int,
         loudness decimal(5,3),
         mode int,
         speechiness decimal(5,4),
         acousticness decimal(5,4),
         instrumentalness float,
         liveness decimal(5,4),
         valence decimal(5,4),
         tempo decimal(6,3),
         time_signature int,
         track_genre string
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\n' LOCATION '{}'
TBLPROPERTIES ('compressionType'='gzip', 'skip.header.line.count'='1')""".format(
    database_name, table_name_csv, s3_private_path_csv
)

print(statement)

In [ ]:
#yyyyyyyyyyyyyyyyyyyyyy
import pandas as pd

# Define your query
query = f"""
    SELECT *
    FROM {database_name}.{table_name_csv}
    LIMIT 10
"""

# Read the query result into a DataFrame
df = pd.read_sql(query, conn)

# Display the DataFrame
print(df.head())

In [ ]:
# Correct columns with voided data for query exercises
import numpy as np
from scipy.stats import skewnorm

# Replace "key" column with random integers from 1 to 11 (inclusive)
df['key'] = np.random.randint(1, 12, size=len(df))

# Fill 'explicit' column with 0s and 1s — 98% 0s, 2% 1s
df['explicit'] = np.random.choice([0, 1], size=len(df), p=[0.98, 0.02])

# Generate normally distributed values centered at 0.5 with std dev of 0.1
danceability = np.random.normal(loc=0.5, scale=0.1, size=len(df))

# Clip values to be within [0, 1] and round to 3 decimal places
df['danceability'] = np.clip(danceability, 0, 1).round(3)

df["popularity"] = df["duration_ms"]
df["liveness"] = df["tempo"]
df["time_signature"] = df["track_genre"]
df["loudness"] = df["mode"]

df["mode"] = df["speechiness"].astype(float).astype(int)

df['speechiness'] = np.random.uniform(0.01, 0.1, size=len(df))

# Parameters for skewed normal
size = len(df)                     # Number of rows in your DataFrame
skewness = -5                      # Negative = left skew (more values above 250,000)
mean = 250000
scale = 30000                     # Spread of the distribution

# Generate skewed data
skewed_data = skewnorm.rvs(a=skewness, loc=mean, scale=scale, size=size)

# Clip values to stay within [100000, 350000]
skewed_data = np.clip(skewed_data, 100000, 350000).astype(int)

# Assign to 'duration_ms' column
df["duration_ms"] = skewed_data

genres = [
    "acoustic", "black-metal", "brazil", "chill", "comedy", "dance-hall", "detroit-techno",
    "drum-and-bass", "edm", "french", "german", "guitar", "house", "idm", "kids", "latino,metal",
    "progressive-house", "r-n-b", "reggaeton", "rockabilly", "samba", "show-tunes", "sleep",
    "spanish", "synth-pop", "world-music"
]

df["track_genre"] = np.random.choice(genres, size=len(df))

# Display the updated DataFrame
print(df.head())